In [2]:
import xml.etree.ElementTree as ElementTree

class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    def __init__(self, parent_element):
        if parent_element.items():
            self.updateShim( dict(parent_element.items()) )
        for element in parent_element:
            if len(element):
                aDict = XmlDictConfig(element)
            #   if element.items():
            #   aDict.updateShim(dict(element.items()))
                self.updateShim({element.tag: aDict})
            elif element.items():    # items() is specialy for attribtes
                elementattrib= element.items()
                if element.text:           
                    elementattrib.append((element.tag,element.text ))     # add tag:text if there exist
                self.updateShim({element.tag: dict(elementattrib)})
            else:
                self.updateShim({element.tag: element.text})

    def updateShim (self, aDict ):
        for key in aDict.keys():   # keys() includes tag and attributes
            if key in self:
                value = self.pop(key)
                if type(value) is not list:
                    listOfDicts = []
                    listOfDicts.append(value)
                    listOfDicts.append(aDict[key])
                    self.update({key: listOfDicts})
                else:
                    value.append(aDict[key])
                    self.update({key: value})
            else:
                self.update({key:aDict[key]})



In [3]:
!pip install xmltodict

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
import xml.etree.ElementTree as ElementTree

import xmltodict
import pprint
import json
import pandas as pd
import numpy as np
from datetime import date



#tree = ElementTree.parse('sample.xml')
tree = ElementTree.parse('/Users/mina/Documents/NYU/BUILT/Matsim_Intermodal_comparison/raw/BUILT.30.experienced_plans.xml')
root = tree.getroot()
xmldict = XmlDictConfig(root)

#xmldict['person']


## Teresa: use xml object function to reshape data efficiently
-ref: https://stackoverflow.com/questions/1912434/how-do-i-parse-xml-in-python

In [5]:
#initiate activity related list object 
id_data_act = []
activity_data = []
start_time = []
end_time = []
for type_tag in root.findall('person'):
    for act in type_tag.findall('plan/activity'):
        id_data_act.append(type_tag.get('id')) #getting id
        activity_data.append(act.get('type'))  #getting work, home 
        start_time.append(act.get('start_time'))
        end_time.append(act.get('end_time'))


In [6]:
#initiate mode related list object 

id_data_mode = []
mode_data = []

for type_tag in root.findall('person'):
    if len(type_tag.findall('plan/leg')) == 0:
        id_data_mode.append(type_tag.get('id')) 
        mode_data.append('na')
 
    for i ,mode in enumerate(type_tag.findall('plan/leg')):
        id_data_mode.append(type_tag.get('id')) 
        mode_data.append(mode.get('mode'))
        if i == len(type_tag.findall('plan/leg'))-1:
            id_data_mode.append(type_tag.get('id')) 
            mode_data.append('N/A')

In [13]:
actList =  list(zip(id_data_act, activity_data, start_time, end_time))
modeList = list(zip(mode_data))
df_act = pd.DataFrame(actList, columns=['id','activity','start_time','end_time'])
df_mode = pd.DataFrame(modeList, columns=['mode'])


In [14]:
df_merged = pd.concat([df_act, df_mode],axis = 1)

In [15]:
df_merged.head(30)

,id,activity,start_time,end_time,mode
0,0,Work,None,09:03:00,bike
1,0,Work,10:21:16,13:31:16,bike
2,0,Secondary,13:36:27,13:42:27,bike
3,0,Work,13:52:13,18:52:13,bike
4,0,Home,20:12:56,None,N/A
5,1,Home,None,06:03:00,car
6,1,Work,06:24:23,11:53:23,N/A
7,1007981,Home,None,07:52:00,walk
8,1007981,Secondary,08:14:39,08:19:39,transit_walk
9,1007981,Work,12:37:14,22:54:14,N/A


In [29]:
# df_merged['start_time']=pd.to_datetime(df_merged['start_time'])
# df_merged['end_time']=pd.to_datetime(df_merged['end_time'])


# df_merged['start_time']=pd.to_datetime(df_merged['start_time'],format= '%H:%M:%S' ).dt.time
# df_merged['end_time']=pd.to_datetime(df_merged['end_time'],format= '%H:%M:%S' ).dt.time


import datetime as dt


df_merged['start_time']=dt.datetime.strptime(df_merged['start_time'],'%H:%M:%S')
df_merged['end_time']=dt.datetime.strptime(df_merged['end_time'],'%H:%M:%S')
diff = (df_merged['end_time'] - df_merged['start_time']) 
diff.seconds/60 

df_merged.head()


TypeError: strptime() argument 1 must be str, not Series

In [18]:
df_merged.iloc[7]['id']


'1007981'

In [32]:
# check travel time
from datetime import date

trave_time = pd.DataFrame(columns={'id','travel_time'})

for i in range(0,len(df_merged)):
    if( df_merged.iloc[i]['start_time']!=None):
        df_merged.iloc[i]['start_time']=dt.datetime.strptime(df_merged.iloc[i]['start_time'],'%H:%M:%S')
    if( df_merged.iloc[i]['end_time']!=None):
        df_merged.iloc[i]['end_time']=dt.datetime.strptime( df_merged.iloc[i]['end_time'],'%H:%M:%S')
    
    if(df_merged.iloc[i]['id']==df_merged.iloc[i+1]['id']):
        diff= df_merged.iloc[i]['end_time']  - df_merged.iloc[i+1]['start_time']
        #diff =pd.Timedelta(diff).seconds/3600
    else:
        diff = np.nan
    
    trave_time=trave_time.append([{'id': df_merged.iloc[i]['id'], 'travel_time':diff}], ignore_index=True)




TypeError: strptime() argument 1 must be str, not NaTType

In [26]:
#merge 

df_final =  pd.concat([df_merged, trave_time['travel_time']], axis=1)
df_final

,id,activity,start_time,end_time,mode,travel_time
0,0,Work,NaT,09:03:00,bike,NaN
1,0,Work,10:21:16,13:31:16,bike,NaN
2,0,Secondary,13:36:27,13:42:27,bike,NaN
3,0,Work,13:52:13,18:52:13,bike,NaN
4,0,Home,20:12:56,NaT,N/A,NaN
5,1,Home,NaT,06:03:00,car,NaN
6,1,Work,06:24:23,11:53:23,N/A,NaN
7,1007981,Home,NaT,07:52:00,walk,NaN
8,1007981,Secondary,08:14:39,08:19:39,transit_walk,NaN
9,1007981,Work,12:37:14,22:54:14,N/A,NaN


In [28]:
#group by id for travel_time

####### add mode on the final group by
total_travel_time = df_final.groupby(['id']).sum()
total_travel_time

#len(travel_time_comparison)

,travel_time
id,
0,93.101111
1,23.643611
1007981,43.329444
1007982,13.086667
1007983,42.995278
1007984,42.589167
1007985,13.489722
1007986,34.708889
1007987,34.830000


In [29]:
df_final.to_excel("10th_mode_final.xlsx")



ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
total_travel_time.to_excel("10th_totalTime_final.xlsx")
#df_merged.to_csv("sample2.csv",sep='\t', encoding='utf-8')
print("all done")